http://seananderson.ca/2013/10/19/reshape.html

# Tabular Data Gym 2. Reshaping.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style
matplotlib.style.use('ggplot')

%matplotlib inline

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(magrittr)
library(ggplot2)
library(dplyr)
library(reshape2)

In [ ]:
%%R
names(airquality) <- tolower(names(airquality))
head(airquality)

In [ ]:
aq_pd = %R airquality
aq_pd.head()

Since `rpy2` does not treat NA well, we need to change it manually to `np.nan`.

In [ ]:
def fix_nans_in_df(colname, df):
    df.ix[df[colname] < 0, colname] = np.NaN    

for colname in aq_pd.columns.values:
    fix_nans_in_df(colname, aq_pd)

aq_pd.head()

## Melt/stack: changing wide format into long

R function `melt` from `reshape2` package change wide fromat into long. 

In [ ]:
%%R

aql <- melt(airquality)
aql %>% head

### Question 1

One can get similar result using pandas' function `melt` on object `aq_pd`. 

### Question 2

In [ ]:
%%R
airquality %>%
    melt(id.vars = c("month", "day")) %>%
    head()

Now let's try to get it with pandas.

## Question 3 Putting names to columns

In [ ]:
%%R

aql <-
  melt(airquality, id.vars = c("month", "day"),
  variable.name = "climate_variable", 
  value.name = "climate_value") %>%
  mutate(date=as.Date(paste0("2015-",month,'-',day)))
head(aql)

Your turn with pandas.

### Comments about ploting

When one uses ggplot it is natural to use long format when ploting.

In [ ]:
%%R
aql %>%
  ggplot() +
  geom_line(aes(date, climate_value, colour=climate_variable))

But with pandas' mathplotlib interface it is easier to use wild format.

First let's create dates.

In [ ]:
aq_pd2= aq_pd.copy()

aq_pd2['date'] = \
    [pd.Timestamp(s) 
     for s in ('2015-'+aq_pd['month'].astype(int).astype(str)+'-'
               +aq_pd['day'].astype(int).astype(str))]

Then selecting columns and adding aproprite index one get: 

In [ ]:
aq_pd2_indexed = aq_pd2[['date', 'ozone', 'solar.r', 'wind', 'temp']]
aq_pd2_indexed.set_index('date')
aq_pd2_indexed.plot()

### Question 4. Time for dcast

In [ ]:
%%R
aqw <- dcast(aql, month + day ~ variable)
head(aqw)

The approprite pandas method seems to be `pivot_table`. Can you try?

### Question 5. Melting with stack

There is another pandas' method for maelting. It is called stack. Can you try it with `aq_pd`? What is the type of the result? Can you make it to be DataFrame with comlumns `varaiable` and `value`?

### Question 6. Melt, filter, group, agregate.

In [ ]:
%%R
aql %>% head

In [ ]:
%%R
airquality %>%
    melt(id.vars = c("month", "day")) %>%
    filter(!is.na(value)) %>%
    group_by(variable) %>%
    summarize(total= sum(value))  

In [ ]:
%%R
library(data.table)
data.table(airquality) %>%
    melt(id.vars = c("month", "day")) %>%
    .[!is.na(value), .(total=sum(value)), by=variable]

Time for doing this with pands: